In [ ]:
!pip install numpy
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install pytorch_lightning==0.8.4
!pip install comet-ml
!pip install torchsummary

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 307kB 8.7MB/s 
     |████████████████████████████████| 829kB 19.9MB/s 
     |████████████████████████████████| 276kB 29.9MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=242494ae1c679c565a0551f24f2d79aeebe89e1942fccef7fddb22332239ed87
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=8a56c9b45b78766e9fad340185c71091f8d0654b4d4bf2647865cfb297fab164
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built future PyYAML
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Su

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Sep  1 20:25:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Mount your personal google drive
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
cd /content/drive/My\ Drive/KBP_Challenge/apex
ls
pip install -v --no-cache-dir ./

Writing setup.sh


In [ ]:
# !sh setup.sh

In [ ]:
primary_directory = '/content/drive/My Drive/KBP_Challenge'
%cd /content/drive/My\ Drive/KBP_Challenge
%ls

/content/drive/My Drive/KBP_Challenge
apex/         data/            pretrained_models/  setup.sh  study.pkl
checkpoints/  lightning_logs/  provided_code/      src/


In [ ]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateLogger
from pytorch_lightning.logging import CometLogger
from src.models.pix2pix_model import Pix2PixModel
from pytorch_lightning.callbacks import ModelCheckpoint
from src.options.train_options import TrainOptions
from torchsummary import summary

In [ ]:
args = ['--batchSize', '8',
        '--primary_directory', primary_directory,
        '--which_model_netG', 'unet_128_3d',
        '--which_model_netD', 'n_layers_3d',
        '--n_layers_D', '3',
        '--num_D', '1',
        '--norm_D', 'spectralbatch',
        '--norm_G', 'spectralbatch',
        '--resnet_depth', '18',
        '--which_direction', 'AtoB',
        '--input_nc', '1',
        '--lambda_A', '10',
        '--lr_policy', 'plateau',
        '--epoch_count', '300',
        '--load_epoch', '-1',
        '--lr_D', '0.0004',
        '--lr_G', '0.0001',
        '--lr_max', '0.01',
        '--lr_step_size', '25',
        '--loss_function', 'smoothed_L1',
        '--init_type', 'xavier',
        #'--no_scaling',
        # '--no_normalization',
        '--no_perceptual_loss',
        '--patience', '5',
        '--n_critic', '1',
        '--n_generator', '3',
        '--cut_blur_mask', '50',
        '--weight_cliping_limit', '0.1',
        '--inst_noise_sigma', '80',
        '--inst_noise_sigma_iters', '100']

opt = TrainOptions().parse(args)

------------ Options -------------
batchSize: 8
beta1: 0.5
beta2: 0.999
checkpoints_dir: ./checkpoints
continue_train: False
cut_blur_mask: 50
dataset_mode: unaligned
display_freq: 200
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 128
epoch_count: 300
fineSize: 128
gpu_ids: -1
init_type: xavier
input_nc: 1
inst_noise_sigma: 80.0
inst_noise_sigma_iters: 100
isTrain: True
lambda_A: 10.0
lambda_B: 10.0
lambda_identity: 0.5
lambda_perceptual: 10.0
loadSize: 128
load_epoch: -1
loss_function: smoothed_L1
lr_D: 0.0004
lr_G: 0.0001
lr_decay_iters: 50
lr_max: 0.01
lr_policy: plateau
lr_step_size: 25
max_dataset_size: inf
model: pix2pix
nThreads: 2
n_critic: 1
n_generator: 3
n_layers_D: 3
name: experiment_name
ndf: 64
ngf: 64
niter: 100
niter_decay: 100
no_augment: False
no_dropout: False
no_flip: False
no_html: False
no_img: False
no_lsgan: False
no_normalization: False
no_perceptual_loss: True
no_scaling: False
norm: batch_3d
norm_D: spectralbatch
norm_G: spect

In [ ]:
model = Pix2PixModel(opt, model_name='train_1', stage='training')
summary(model.generator.to("cuda"), (1, 128, 128, 128))

Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization


/content/drive/My Drive/KBP_Challenge/src/models/networks.py:39: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight.data, gain=0.02)


Using spectral normalization
Using spectral normalization
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 64, 64, 64]           4,096
         LeakyReLU-2       [-1, 64, 64, 64, 64]               0
            Conv3d-3      [-1, 128, 32, 32, 32]         524,288
       BatchNorm3d-4      [-1, 128, 32, 32, 32]             256
         LeakyReLU-5      [-1, 128, 32, 32, 32]               0
            Conv3d-6      [-1, 256, 16, 16, 16]       2,097,152
       BatchNorm3d-7      [-1, 256, 16, 16, 16]             512
         LeakyReLU-8      [-1, 256, 16, 16, 16]               0
            Conv3d-9         [-1, 512, 8, 8, 8]       8,388,608
      BatchNorm3d-10         [-1, 512, 8, 8, 8]           1,024
        LeakyReLU-11         [-1, 512, 8, 8, 8]               0
           Conv3d-12         [-1, 512, 4, 4, 4]      16,777,216
      BatchNorm3d-13         [-1, 512, 4, 4, 

In [ ]:
checkpoints_dir = '{}/checkpoints'.format(primary_directory)
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoints_dir,
    verbose=True,
    save_last=False,
    save_top_k=0,
    monitor='dose_score',
    mode='min'
)
lr_logger = LearningRateLogger()

comet_logger = CometLogger(
    api_key="eyAsnp1KA7fXLxFMkEWKjhygS",
    project_name="kbp-challenge",
    workspace="mahruskazi"
)

CometLogger will be initialized in online mode


In [ ]:
checkpoint = None
if opt.load_epoch != -1:
  checkpoint = '{}/epoch={}.ckpt'.format(checkpoints_dir, opt.load_epoch)

trainer = pl.Trainer(logger=comet_logger,
                     resume_from_checkpoint=checkpoint,
                     gpus=1,
                     checkpoint_callback=checkpoint_callback,
                     callbacks=[lr_logger],
                     max_epochs=opt.epoch_count,
                     check_val_every_n_epoch=1,
                     num_sanity_val_steps=10,
                     limit_val_batches=1.0,
                     accumulate_grad_batches=1,
                     gradient_clip_val=opt.weight_cliping_limit,
                     weights_summary='full')
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `test_dataloader()` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/mahruskazi/kbp-challenge/8d0f83d4e41c4755a11a25abb60a9a4f


   | Name                                                                    | Type                    | Params
---------------------------------------------------------------------------------------------------------------------
0  | generator                                                               | UnetGenerator           | 167 M 
1  | generator.model                                                         | UnetSkipConnectionBlock | 167 M 
2  | generator.model.model                  

Number of validation patients: 50


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


100%|██████████| 10/10 [00:12<00:00,  1.26s/it]

{'log': {'dose_score': tensor(25.2296, device='cuda:0', dtype=torch.float64), 'dvh_score': tensor(19.5934, device='cuda:0', dtype=torch.float64)}}
Number of training patients: 150
Number of validation patients: 50


...

{'log': {'dose_score': tensor(7.9758, device='cuda:0', dtype=torch.float64), 'dvh_score': tensor(12.1953, device='cuda:0', dtype=torch.float64)}}


100%|██████████| 50/50 [00:36<00:00,  1.36it/s]


{'log': {'dose_score': tensor(7.2979, device='cuda:0', dtype=torch.float64), 'dvh_score': tensor(12.2448, device='cuda:0', dtype=torch.float64)}}


100%|██████████| 50/50 [00:37<00:00,  1.34it/s]


{'log': {'dose_score': tensor(7.4948, device='cuda:0', dtype=torch.float64), 'dvh_score': tensor(12.5909, device='cuda:0', dtype=torch.float64)}}
